In [9]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse, mae

# Učitavanje podataka
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

# Uklanjanje redova sa NaN vrednostima u ocenama
rating_df.dropna(subset=['rating'], inplace=True)

# Priprema podataka za Surprise biblioteku
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Podela podataka na trenig i test setove
trainset, testset = train_test_split(data, test_size=0.2)

# Kreiranje SVD modela
svd = SVD()

# Treniranje modela
svd.fit(trainset)

# Evaluacija modela
predictions = svd.test(testset)

# Računanje RMSE i MAE metrika
print(f'RMSE: {rmse(predictions)}')
print(f'MAE: {mae(predictions)}')

# Funkcija za preporuke bazirane na SVD modelu
def get_svd_recommendations(user_id, n=5):
    user_ratings = rating_df[rating_df['user_id'] == user_id]
    already_rated = user_ratings['anime_id'].tolist()
    
    all_anime_ids = anime_df['anime_id'].tolist()
    recommendations = []
    
    for anime_id in all_anime_ids:
        if anime_id not in already_rated:
            pred = svd.predict(user_id, anime_id)
            recommendations.append((anime_id, pred.est))
    
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    top_n_recommendations = recommendations[:n]
    recommended_anime_names = [anime_df.loc[anime_df['anime_id'] == anime_id, 'name'].values[0] for anime_id, _ in top_n_recommendations]
    
    return recommended_anime_names

# Primer dobijanja preporuka za korisnika sa ID-jem 23
user_id = 23
recommendations = get_svd_recommendations(user_id)
print(f'Preporuke za korisnika {user_id}: {recommendations}')


RMSE: 2.1682
RMSE: 2.1681887571160323
MAE:  1.5484
MAE: 1.548384517902984
Preporuke za korisnika 23: ['Kimi no Na wa.', 'Fullmetal Alchemist: Brotherhood', 'Gintama°', 'Steins;Gate', 'Gintama&#039;']


In [6]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from surprise.accuracy import rmse, mae
import numpy as np

# Učitavanje podataka
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

# Pretprocesiranje podataka
rating_df.drop_duplicates(inplace=True)
rating_df = rating_df[rating_df['rating'] > 0]  # Uklonili smo nevalidne ocene

# Učitavanje podataka u Surprise dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Podela podataka na trening i test skupove
trainset, testset = train_test_split(data, test_size=0.2)

# Treniranje SVD modela
svd = SVD()
svd.fit(trainset)

# Evaluacija modela
predictions = svd.test(testset)
#print(f'RMSE pre optimizacije: {rmse(predictions):.4f}')
#print(f'MAE pre optimizacije: {mae(predictions):.4f}')

# Funkcija za dobijanje preporuka za korisnika koristeći SVD model
def get_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Funkcija za pronalaženje sličnih animea na osnovu žanra
def find_similar_anime(anime_id, anime_ids_watched, n=5):
    if anime_id not in anime_df.index:
        return []

    genres = anime_df.loc[anime_id, 'genre']
    if pd.notnull(genres) and isinstance(genres, str):
        genres_list = genres.split(', ')
        similar_anime = anime_df[anime_df['genre']
                                 .apply(lambda x: isinstance(x, str) and any(genre in x.split(', ') 
                                                                             for genre in genres_list))]
    else:
        similar_anime = pd.DataFrame()

    # Isključivanje već gledanih animea
    similar_anime = similar_anime[~similar_anime.index.isin(anime_ids_watched)].head(n)
    return similar_anime.index.tolist()

# Funkcija za dobijanje sadržajno zasnovanih preporuka
def get_content_based_recommendations(user_id, k=5):
    anime_ids_watched = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    content_based_recommendations = []
    for anime_id in anime_ids_watched:
        similar_anime_ids = find_similar_anime(anime_id, anime_ids_watched)
        content_based_recommendations.extend(similar_anime_ids)

    anime_names = [anime_df.loc[idx, 'name'] for idx in content_based_recommendations if idx in anime_df.index]
    return anime_names[:k]

# Funkcija za kombinovane preporuke
def get_combined_recommendations(user_id, n=5):
    svd_recommendations = get_svd_recommendations(user_id, n=n)
    content_based_recommendations = get_content_based_recommendations(user_id, k=n)

    combined_recommendations = list(set(svd_recommendations + content_based_recommendations))[:n]
    return combined_recommendations

# Evaluacija Precision@k i Recall@k
def precision_recall_at_k(predictions, k=10, threshold=7.0):
    user_est_true = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))

    precisions = []
    recalls = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 0)

    precision = np.mean(precisions)
    recall = np.mean(recalls)
    return precision, recall

# Evaluacija pre optimizacije
precision, recall = precision_recall_at_k(predictions, k=5, threshold=7.0)
print(f'Precision (SVD, pre optimizacije): {precision:.4f}')
print(f'Recall (SVD, pre optimizacije): {recall:.4f}')

# Treniranje modela sa najboljim parametrima
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Najbolji parametri
print(f'Najbolji parametri: {gs.best_params["rmse"]}')

best_svd = gs.best_estimator['rmse']
best_svd.fit(trainset)

# Evaluacija modela sa najboljim parametrima
predictions = best_svd.test(testset)
print(f'RMSE posle optimizacije: {rmse(predictions):.4f}')
print(f'MAE posle optimizacije: {mae(predictions):.4f}')

precision, recall = precision_recall_at_k(predictions, k=5, threshold=7.0)
print(f'Precision (SVD, posle optimizacije): {precision:.4f}')
print(f'Recall (SVD, posle optimizacije): {recall:.4f}')

# Funkcija za preporuke sa najboljim SVD modelom
def get_best_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [best_svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Primer dobijanja preporuka za korisnika sa ID-jem 815
user_id = 815
svd_recommendations = get_svd_recommendations(user_id)
content_based_recommendations = get_content_based_recommendations(user_id)
combined_recommendations = get_combined_recommendations(user_id)

# Ispis preporuka pre optimizacije
print(f'\nPreporuke za korisnika {user_id} koristeći SVD pre optimizacije:')
for i, anime in enumerate(svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju pre optimizacije:')
for i, anime in enumerate(content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} pre optimizacije:')
for i, anime in enumerate(combined_recommendations, 1):
    print(f'{i}. {anime}')

# Ispis preporuka posle optimizacije
best_svd_recommendations = get_best_svd_recommendations(user_id)
best_content_based_recommendations = get_content_based_recommendations(user_id)
best_combined_recommendations = get_combined_recommendations(user_id)

print(f'\nPreporuke za korisnika {user_id} koristeći SVD sa najboljim parametrima:')
for i, anime in enumerate(best_svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju sa najboljim parametrima:')
for i, anime in enumerate(best_content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} sa najboljim parametrima:')
for i, anime in enumerate(best_combined_recommendations, 1):
    print(f'{i}. {anime}')


RMSE: 1.2166
RMSE pre optimizacije: 1.2166
MAE:  0.9199
MAE pre optimizacije: 0.9199
Precision (SVD, pre optimizacije): 0.9438
Recall (SVD, pre optimizacije): 0.5539
Najbolji parametri: {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1}
RMSE: 1.1970
RMSE posle optimizacije: 1.1970
MAE:  0.9046
MAE posle optimizacije: 0.9046
Precision (SVD, posle optimizacije): 0.9471
Recall (SVD, posle optimizacije): 0.5542

Preporuke za korisnika 815 koristeći SVD pre optimizacije:
1. Mononoke Hime
2. Rainbow: Nisha Rokubou no Shichinin
3. Steins;Gate
4. Clannad: After Story
5. Gintama&#039;

Preporuke za korisnika 815 koristeći filtriranje zasnovano na sadržaju pre optimizacije:
1. Kimi no Na wa.
2. Fullmetal Alchemist: Brotherhood
3. Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou
4. Ginga Eiyuu Densetsu
5. Clannad: After Story

Kombinovane preporuke za korisnika 815 pre optimizacije:
1. Kimi no Na wa.
2. Clannad: After Story
3. Rainbow: Nisha Rokubou no Shichinin
4. Fullme

In [2]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from surprise.accuracy import rmse, mae
import numpy as np

# Učitavanje podataka
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

# Pretprocesiranje podataka
rating_df.drop_duplicates(inplace=True)
rating_df = rating_df[rating_df['rating'] > 0]  # Uklonili smo nevalidne ocene

# Učitavanje podataka u Surprise dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Podela podataka na trening i test skupove
trainset, testset = train_test_split(data, test_size=0.2)

# Treniranje SVD modela
svd = SVD()
svd.fit(trainset)

# Evaluacija modela
predictions = svd.test(testset)
print(f'RMSE pre optimizacije: {rmse(predictions):.4f}')
print(f'MAE pre optimizacije: {mae(predictions):.4f}')

# Funkcija za dobijanje preporuka za korisnika koristeći SVD model
def get_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Funkcija za pronalaženje sličnih animea na osnovu žanra
def find_similar_anime(anime_id, anime_ids_watched, n=5):
    if anime_id not in anime_df.index:
        return []

    genres = anime_df.loc[anime_id, 'genre']
    if pd.notnull(genres) and isinstance(genres, str):
        genres_list = genres.split(', ')
        similar_anime = anime_df[anime_df['genre']
                                 .apply(lambda x: isinstance(x, str) and any(genre in x.split(', ') 
                                                                             for genre in genres_list))]
    else:
        similar_anime = pd.DataFrame()

    # Isključivanje već gledanih animea
    similar_anime = similar_anime[~similar_anime.index.isin(anime_ids_watched)].head(n)
    return similar_anime.index.tolist()

# Funkcija za dobijanje sadržajno zasnovanih preporuka
def get_content_based_recommendations(user_id, k=5):
    anime_ids_watched = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    content_based_recommendations = []
    for anime_id in anime_ids_watched:
        similar_anime_ids = find_similar_anime(anime_id, anime_ids_watched)
        content_based_recommendations.extend(similar_anime_ids)

    anime_names = [anime_df.loc[idx, 'name'] for idx in content_based_recommendations if idx in anime_df.index]
    return anime_names[:k]

# Funkcija za kombinovane preporuke
def get_combined_recommendations(user_id, n=5):
    svd_recommendations = get_svd_recommendations(user_id, n=n)
    content_based_recommendations = get_content_based_recommendations(user_id, k=n)

    combined_recommendations = list(set(svd_recommendations + content_based_recommendations))[:n]
    return combined_recommendations

# Evaluacija Precision@k i Recall@k
def precision_recall_at_k(predictions, k=10, threshold=7.0):
    user_est_true = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))

    precisions = []
    recalls = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 0)

    precision = np.mean(precisions)
    recall = np.mean(recalls)
    return precision, recall

# Evaluacija pre optimizacije
precision, recall = precision_recall_at_k(predictions, k=5, threshold=7.0)
print(f'Precision (SVD, pre optimizacije): {precision:.4f}')
print(f'Recall (SVD, pre optimizacije): {recall:.4f}')

# Treniranje modela sa najboljim parametrima
param_grid = {
    'n_factors': [100, 150, 200],
    'n_epochs': [20, 30, 40, 50],
    'lr_all': [0.005, 0.01, 0.015],
    'reg_all': [0.02, 0.1, 0.15]
}


gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Najbolji parametri
print(f'Najbolji parametri: {gs.best_params["rmse"]}')

best_svd = gs.best_estimator['rmse']
best_svd.fit(trainset)

# Evaluacija modela sa najboljim parametrima
predictions = best_svd.test(testset)
print(f'RMSE posle optimizacije: {rmse(predictions):.4f}')
print(f'MAE posle optimizacije: {mae(predictions):.4f}')

precision, recall = precision_recall_at_k(predictions, k=5, threshold=7.0)
print(f'Precision (SVD, posle optimizacije): {precision:.4f}')
print(f'Recall (SVD, posle optimizacije): {recall:.4f}')

# Funkcija za preporuke sa najboljim SVD modelom
def get_best_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [best_svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Primer dobijanja preporuka za korisnika sa ID-jem 815
user_id = 815
svd_recommendations = get_svd_recommendations(user_id)
content_based_recommendations = get_content_based_recommendations(user_id)
combined_recommendations = get_combined_recommendations(user_id)

# Ispis preporuka pre optimizacije
print(f'\nPreporuke za korisnika {user_id} koristeći SVD pre optimizacije:')
for i, anime in enumerate(svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju pre optimizacije:')
for i, anime in enumerate(content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} pre optimizacije:')
for i, anime in enumerate(combined_recommendations, 1):
    print(f'{i}. {anime}')

# Ispis preporuka posle optimizacije
best_svd_recommendations = get_best_svd_recommendations(user_id)
best_content_based_recommendations = get_content_based_recommendations(user_id)
best_combined_recommendations = get_combined_recommendations(user_id)

print(f'\nPreporuke za korisnika {user_id} koristeći SVD sa najboljim parametrima:')
for i, anime in enumerate(best_svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju sa najboljim parametrima:')
for i, anime in enumerate(best_content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} sa najboljim parametrima:')
for i, anime in enumerate(best_combined_recommendations, 1):
    print(f'{i}. {anime}')


RMSE: 1.2037
RMSE pre optimizacije: 1.2037
MAE:  0.9150
MAE pre optimizacije: 0.9150
Precision (SVD, pre optimizacije): 0.9408
Recall (SVD, pre optimizacije): 0.5461
Najbolji parametri: {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.015, 'reg_all': 0.15}
RMSE: 1.1862
RMSE posle optimizacije: 1.1862
MAE:  0.9001
MAE posle optimizacije: 0.9001
Precision (SVD, posle optimizacije): 0.9384
Recall (SVD, posle optimizacije): 0.5436

Preporuke za korisnika 815 koristeći SVD pre optimizacije:
1. Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare
2. Gintama&#039;: Enchousen
3. Clannad: After Story
4. Monster
5. Hunter x Hunter (2011)

Preporuke za korisnika 815 koristeći filtriranje zasnovano na sadržaju pre optimizacije:
1. Kimi no Na wa.
2. Fullmetal Alchemist: Brotherhood
3. Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou
4. Ginga Eiyuu Densetsu
5. Clannad: After Story

Kombinovane preporuke za korisnika 815 pre optimizacije:
1. Fullmetal Alchemist: Brotherhood
2. Hunter x Hunter (

In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.accuracy import rmse, mae
import numpy as np

# Učitavanje podataka
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

# Pretprocesiranje podataka
rating_df.drop_duplicates(inplace=True)
rating_df = rating_df[rating_df['rating'] > 0]  # Uklonili smo nevalidne ocene

# Učitavanje podataka u Surprise dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Podela podataka na trening i test skupove
trainset, testset = train_test_split(data, test_size=0.2)

# Treniranje SVD modela
svd = SVD()
svd.fit(trainset)

# Evaluacija modela
predictions = svd.test(testset)
print(f'RMSE pre optimizacije: {rmse(predictions):.4f}')
print(f'MAE pre optimizacije: {mae(predictions):.4f}')

# Funkcija za dobijanje preporuka za korisnika koristeći SVD model
def get_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Funkcija za pronalaženje sličnih animea na osnovu žanra
def find_similar_anime(anime_id, anime_ids_watched, n=5):
    if anime_id not in anime_df.index:
        return []

    genres = anime_df.loc[anime_id, 'genre']
    if pd.notnull(genres) and isinstance(genres, str):
        genres_list = genres.split(', ')
        similar_anime = anime_df[anime_df['genre']
                                 .apply(lambda x: isinstance(x, str) and any(genre in x.split(', ') 
                                                                             for genre in genres_list))]
    else:
        similar_anime = pd.DataFrame()

    # Isključivanje već gledanih animea
    similar_anime = similar_anime[~similar_anime.index.isin(anime_ids_watched)].head(n)
    return similar_anime.index.tolist()

# Funkcija za dobijanje sadržajno zasnovanih preporuka
def get_content_based_recommendations(user_id, k=5):
    anime_ids_watched = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    content_based_recommendations = []
    for anime_id in anime_ids_watched:
        similar_anime_ids = find_similar_anime(anime_id, anime_ids_watched)
        content_based_recommendations.extend(similar_anime_ids)

    anime_names = [anime_df.loc[idx, 'name'] for idx in content_based_recommendations if idx in anime_df.index]
    return anime_names[:k]

# Funkcija za kombinovane preporuke
def get_combined_recommendations(user_id, n=5):
    svd_recommendations = get_svd_recommendations(user_id, n=n)
    content_based_recommendations = get_content_based_recommendations(user_id, k=n)

    combined_recommendations = list(set(svd_recommendations + content_based_recommendations))[:n]
    return combined_recommendations

# Evaluacija Precision@k i Recall@k
def precision_recall_at_k(predictions, k=10, threshold=7.0):
    user_est_true = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))

    precisions = []
    recalls = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 0)

    precision = np.mean(precisions)
    recall = np.mean(recalls)
    return precision, recall

# Treniranje modela sa najboljim parametrima
param_grid = {
    'n_factors': [100, 150, 200],
    'n_epochs': [20, 30, 40, 50],
    'lr_all': [0.005, 0.01, 0.015],
    'reg_all': [0.02, 0.1, 0.15]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Najbolji parametri
print(f'Najbolji parametri: {gs.best_params["rmse"]}')

best_svd = gs.best_estimator['rmse']
best_svd.fit(trainset)

# Evaluacija modela sa najboljim parametrima
predictions = best_svd.test(testset)
print(f'RMSE posle optimizacije: {rmse(predictions):.4f}')
print(f'MAE posle optimizacije: {mae(predictions):.4f}')

# Evaluacija Precision@k i Recall@k posle optimizacije
precision, recall = precision_recall_at_k(predictions, k=5, threshold=7.0)
print(f'Precision (SVD, posle optimizacije): {precision:.4f}')
print(f'Recall (SVD, posle optimizacije): {recall:.4f}')

# Funkcija za preporuke sa najboljim SVD modelom
def get_best_svd_recommendations(user_id, n=5):
    anime_ids = anime_df['anime_id'].unique()
    user_rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    anime_to_predict = [anime_id for anime_id in anime_ids if anime_id not in user_rated_anime]

    predictions = [best_svd.predict(user_id, anime_id) for anime_id in anime_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    recommended_anime = [anime_df.loc[anime_df['anime_id'] == pred.iid, 'name'].values[0] for pred in predictions[:n]]
    return recommended_anime

# Primer dobijanja preporuka za korisnika sa ID-jem 815
user_id = 815
svd_recommendations = get_svd_recommendations(user_id)
content_based_recommendations = get_content_based_recommendations(user_id)
combined_recommendations = get_combined_recommendations(user_id)

# Ispis preporuka pre optimizacije
print(f'\nPreporuke za korisnika {user_id} koristeći SVD pre optimizacije:')
for i, anime in enumerate(svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju pre optimizacije:')
for i, anime in enumerate(content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} pre optimizacije:')
for i, anime in enumerate(combined_recommendations, 1):
    print(f'{i}. {anime}')

# Ispis preporuka posle optimizacije
best_svd_recommendations = get_best_svd_recommendations(user_id)
best_content_based_recommendations = get_content_based_recommendations(user_id)
best_combined_recommendations = get_combined_recommendations(user_id)

print(f'\nPreporuke za korisnika {user_id} koristeći SVD sa najboljim parametrima:')
for i, anime in enumerate(best_svd_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nPreporuke za korisnika {user_id} koristeći filtriranje zasnovano na sadržaju sa najboljim parametrima:')
for i, anime in enumerate(best_content_based_recommendations, 1):
    print(f'{i}. {anime}')

print(f'\nKombinovane preporuke za korisnika {user_id} sa najboljim parametrima:')
for i, anime in enumerate(best_combined_recommendations, 1):
    print(f'{i}. {anime}')


RMSE: 1.2352
RMSE pre optimizacije: 1.2352
MAE:  0.9321
MAE pre optimizacije: 0.9321
Najbolji parametri: {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.015, 'reg_all': 0.15}
RMSE: 1.2098
RMSE posle optimizacije: 1.2098
MAE:  0.9149
MAE posle optimizacije: 0.9149
Precision (SVD, posle optimizacije): 0.9509
Recall (SVD, posle optimizacije): 0.5626

Preporuke za korisnika 815 koristeći SVD pre optimizacije:
1. Rainbow: Nisha Rokubou no Shichinin
2. Hunter x Hunter (2011)
3. Great Teacher Onizuka
4. Steins;Gate
5. Gintama

Preporuke za korisnika 815 koristeći filtriranje zasnovano na sadržaju pre optimizacije:
1. Kimi no Na wa.
2. Fullmetal Alchemist: Brotherhood
3. Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou
4. Ginga Eiyuu Densetsu
5. Clannad: After Story

Kombinovane preporuke za korisnika 815 pre optimizacije:
1. Fullmetal Alchemist: Brotherhood
2. Gintama
3. Hunter x Hunter (2011)
4. Great Teacher Onizuka
5. Clannad: After Story

Preporuke za korisnika 815 koristeći SVD s